# The Movie Database: Data Extraction & Cleaning

In [1]:
# %pip install cpi

In [2]:
# Import dependencies
import pandas as pd
import json
import requests
from pprint import pprint
import numpy as np
import cpi

from datetime import datetime

# Import config
from config import api_key, db_user, db_password, db_host, db_port, db_name

In [3]:
cpi.update()

### Testing:

# FUNCTIONS:

In [4]:
# Start Timer Function (check on API call performance)
def start_timer():
    start = datetime.now()
    
    return start

In [5]:
# Stop Timer Function (check on API call performance)
def stop_timer(start):
    end = datetime.now()
    elapsed_time = (end - start)

    print(f"Total Time Elapsed:  {elapsed_time.total_seconds()} seconds")

### API CALLS:

In [6]:
# Returns most popular movies
def get_most_popular_movies(api_key):
    
    movies = []
    
    # Loop through pages to get results for movies
    for x in range(1, 101):
        page_number = x

        # Endpoint for finding most popular movies
        discover_movies = "https://api.themoviedb.org/3/discover/movie"
        most_popular_url = f"{discover_movies}?api_key={api_key}&page={page_number}&sort_by=popularity.desc"
        
        # Most popular movies
        tmdb_response = requests.get(most_popular_url).json()
        results = tmdb_response["results"]

#         for y in range(len(results)):
#             movies.append(results[y]) 
        for result in results:
            movies.append(result)
            
    return movies

In [7]:
# Returns movie details as a list
def get_movie_details(api_key, movie_ids):  
    
    movie_details = []
    
    for movie_id in movie_ids:
        # Endpoint & response
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
        movie_response = requests.get(movie_url).json()

        # Append results to lists
        movie_details.append(movie_response)
        
    return movie_details

In [8]:
# Returns keywords as a list
def get_movie_keywords(api_key, movie_ids):
    keyword_details = []
    
    for movie_id in movie_ids:
        # Get keywords for each movie
        keywords_url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={api_key}"
        keywords_response = requests.get(keywords_url).json()
        keywords_results = keywords_response["keywords"]
        
         # Append results to lists
        keyword_details.append(keywords_response)
        
    return keyword_details

In [9]:
# Returns credits as a list
def get_credits(api_key, movie_ids):
    credit_details = []
    
    for movie_id in movie_ids:
        # Generate movie credits
        movie_credits = f"/movie/{movie_id}/credits"
        credits_url = f"https://api.themoviedb.org/3/{movie_credits}?api_key={api_key}"

        # Get the json response for the credits
        credits_response = requests.get(credits_url).json()
        
        credit_details.append(credits_response)
        
    return credit_details

In [10]:
# Returns certifications as a list (rating: G, PG, etc.)
def get_certifications(api_key, movie_ids):
    certifications = []
    
    for movie_id in movie_ids:
        # Get certifications for each movie
        certification_url = f"https://api.themoviedb.org/3/movie/{movie_id}/release_dates?api_key={api_key}"
        
        certification_response = requests.get(certification_url).json()
        
        certifications.append(certification_response)
        
    return certifications

### GET TITLES & IDS:

In [11]:
# Returns single title for specified index number
def get_title(results, idx):
    movie_title = results[idx]["title"]
    
    return movie_title

In [12]:
# Returns ids list
def get_ids(results):
    movie_ids = []
    for x in results:
        movie_ids.append(x["id"])
        
    return movie_ids

In [13]:
# Returns titles list
def get_titles(results):
    movie_titles = []
    for x in results:
        movie_titles.append(x["title"])
        
    return movie_titles

### EXTRACT NEEDED DETAILS FROM COLUMNS:

In [14]:
# Extract the certification (ratings)
def extract_certification(x):
    name = ""
    
    for i in x:
        # Get ratings for US
        if i['iso_3166_1'] == 'US':
             # Append results to lists
            name = i['release_dates'][0]['certification']
            break
    return name

In [15]:
# Function to get the director out of the crew
def get_director(x):
    names = []
    
    for i in x:
        if i['job'] == 'Director':
            name = i['name']
            names.append(name)
            
    if(names):
        return names
    
    return np.nan

In [16]:
# Function to get the director gender out of the crew
def get_director_gender(x):
    names = []
    
    for i in x:
        if i['job'] == 'Director':
            gender = i['gender']
            names.append(gender)
            
    if(names):
        return names
    
    return np.nan

In [17]:
# Function to get the producers out of the crew
def get_producers(x):
    names = []
    
    for i in x:
        if i['job'] == 'Producer':
            name = i['name']
            names.append(name)
        elif i['job'] == "Executive Producer":
            name = i['name']
            names.append(name)
        elif i['job'] == 'Co-Producer':
            name = i['name']
            names.append(name)
            
    if(names):
        return names
    
    return np.nan

In [18]:
# Function to get the screenplay writers out of the crew
def get_writers(x):
    names = []
    for i in x:
        if i['department'] == 'Writing':
            name = i['name']
            names.append(name)
            
    if(names):
        return names
    
    return np.nan

In [19]:
# Function to create cast list
def get_cast_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        
        # Condition for cast (keep top 5)
        if len(names) > 5:
            names = names[:5]
        return names

    return []

In [20]:
# Function to create lists of each feature
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names
        
    return []

In [21]:
# Function to get production companies
def get_production_companies(x):
    names = []
    if isinstance(x, list):
        for i in x:
            name = i['name']
            names.append(name)
        return names
        
    return []

In [22]:
# Get each production company's country of origin
def get_production_company_country(x):
    names = []
    if isinstance(x, list):
        for i in x:
            country = i['origin_country']
            names.append(country)
        return names
    return []

In [23]:
# Get list of all languages available for film
def get_languages(x):
    names = []
    for i in x:
        name = i['english_name']
#         iso = i['iso_639_1']
        names.append(name)
    
    if(names):
        return names
    
    return np.nan

In [24]:
# Create binary column for foreign language films
def original_language_binary(x):
    if (x == 'en'):
        return 0
    elif (x != 'en'):
        return 1
    elif (x == "" | x == " "):
        return 0
    else:
        return 0

### CLEAN DATA FOR SOUP & CREATE SOUP:

In [25]:
# Convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else: 
            return ''

In [26]:
# Clean the overview column (by converting to lowercase)
def clean_overview(x):
    if isinstance(x, str):
        return x.lower()
    else: 
        return ''

In [27]:
# Create soup of words
def create_soup(x):
    space = ' '
    return (space.join(x['keywords_cleaned']) + space + space.join(x['cast_cleaned']) + space 
            + space.join(x['director_cleaned']) + space + space.join(x['producers_cleaned']) + space 
            + space.join(x['writers_cleaned']) + space + space.join(x['genres_cleaned']) + space 
            + space.join(x['production_companies_cleaned']))

In [28]:
# Create soup of words - 2 keywords, 2 genres & overview
def create_soup_overview(x):
    space = ' '
    return (space.join(x['keywords_cleaned']) + space + space.join(x['keywords_cleaned']) + space 
            + space.join(x['cast_cleaned']) + space + space.join(x['director_cleaned']) + space 
            + space.join(x['producers_cleaned']) + space + space.join(x['writers_cleaned']) + space 
            + space.join(x['genres_cleaned']) + space + space.join(x['genres_cleaned']) + space 
            + space.join(x['production_companies_cleaned']) + space + x['overview_cleaned'])

# CALL API

### Get Most Popular Movies JSON Results:

In [29]:
# Start the timer
start = start_timer()

In [30]:
most_popular_movies = get_most_popular_movies(api_key)
# print(most_popular_movies)

In [31]:
# Stop the timer
stop_timer(start)

Total Time Elapsed:  33.07278 seconds


### List of Movie IDs:

In [32]:
movie_ids = get_ids(most_popular_movies)
# print(movie_ids)

In [33]:
len(movie_ids)

2000

### List of Titles:

In [34]:
titles = get_titles(most_popular_movies)
# print(titles)

In [35]:
len(titles)

2000

### Print Individual Title:

In [36]:
# print(get_title(most_popular_movies, 0))

### Get Movie Details:

In [37]:
# Start the timer
start = start_timer()

In [38]:
details = get_movie_details(api_key, movie_ids)

In [39]:
# print(details)

In [40]:
# Stop the timer
stop_timer(start)

Total Time Elapsed:  259.913294 seconds


### Get Keywords:

In [41]:
# Start the timer
start = start_timer()

In [42]:
keywords = get_movie_keywords(api_key, movie_ids)

In [43]:
# print(keywords)

In [44]:
# Stop the timer
stop_timer(start)

Total Time Elapsed:  262.196792 seconds


### Get Credits:

In [45]:
# Start the timer
start = start_timer()

In [46]:
credits = get_credits(api_key, movie_ids)

In [47]:
# print(credits)

In [48]:
# Stop the timer
stop_timer(start)

Total Time Elapsed:  267.810107 seconds


### Get Certifications:

In [49]:
# Start the timer
start = start_timer()

In [50]:
# Get the certifications for each movie (rating: G, PG, etc.)
certifications = get_certifications(api_key, movie_ids)
# certifications = get_certifications(api_key, [32657, 672582])

In [51]:
# print(certifications)

In [52]:
# Stop the timer
stop_timer(start)

Total Time Elapsed:  265.061754 seconds


## Movie Details DataFrame:

In [53]:
# Convert details to json
json_details_string = json.dumps(details)
# Convert json to dataframe
movie_details_df = pd.read_json(json_details_string)

# Export to save
# movie_details_df.to_csv("./static/data/movie_details.csv", index=False)
movie_details_df.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"{'id': 558216, 'name': 'Venom Collection', 'po...",110000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.venom.movie,580489,tt7097896,en,Venom: Let There Be Carnage,...,2021-09-30,454000000,97,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,,Venom: Let There Be Carnage,False,7.2,3708
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,None,200000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",https://www.netflix.com/us/title/81161626,512195,tt7991608,en,Red Notice,...,2021-11-04,0,116,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Pro and cons.,Red Notice,False,6.8,1616
2,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,None,150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.marvel.com/movies/shang-chi-and-th...,566525,tt9376612,en,Shang-Chi and the Legend of the Ten Rings,...,2021-09-01,430238384,132,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,You can't outrun your destiny.,Shang-Chi and the Legend of the Ten Rings,False,7.9,3844
3,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,None,9100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,589761,tt10648714,ru,Чернобыль,...,2021-04-15,5370393,136,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Chernobyl 1986,False,6.2,214
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,None,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",,585245,tt2397461,en,Clifford the Big Red Dog,...,2021-11-10,40000000,97,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Adventure has never been bigger.,Clifford the Big Red Dog,False,7.6,332


In [54]:
# Extract year from release date
movie_details_df["year"] = pd.DatetimeIndex(movie_details_df['release_date']).year
# Drop any N/A from year
movie_details_df.dropna(inplace=True, how="any", subset=['year','release_date'])
 
# Convert to int
movie_details_df["year"] = movie_details_df["year"].astype(int)
    
# Rename columns to prepare for inflation calculation
movie_details_df = movie_details_df.rename(columns = {
    "revenue": "original_revenue",
    "budget": "original_budget"
})

In [55]:
len(movie_details_df)

1990

## Adjust Budget & Revenue for Inflation

In [56]:
# Create dataframe for cpi
cpi_df = pd.DataFrame(columns=['year', 'cpi_2021', 'cpi_old'])

In [57]:
# Create list of years (1913-2021)
years = []
for i in range(1913, 2022):
    years.append(i)

In [58]:
# Define 2021 cpi
cpi_2021 = 269.489

# Loop through years to append to dataframe
for year in years:
    if year != 2021:
        cpi_old = cpi.get(year)
        cpi_df = cpi_df.append({"year": year, 'cpi_2021': cpi_2021, 'cpi_old': cpi_old}, ignore_index=True)
    elif year == 2021:
        cpi_df = cpi_df.append({"year": year, 'cpi_2021': cpi_2021, 'cpi_old': cpi_2021}, ignore_index=True)

In [59]:
# Convert year type to int
cpi_df['year'] = cpi_df['year'].astype(int)
cpi_df.head()

,year,cpi_2021,cpi_old
0,1913,269.489,9.9
1,1914,269.489,10.0
2,1915,269.489,10.1
3,1916,269.489,10.9
4,1917,269.489,12.8


In [60]:
# Merge movie details df with cpi df
movie_details_df = movie_details_df.merge(cpi_df, on="year")
movie_details_df.head()

,adult,backdrop_path,belongs_to_collection,original_budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cpi_2021,cpi_old
0,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"{'id': 558216, 'name': 'Venom Collection', 'po...",110000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.venom.movie,580489,tt7097896,en,Venom: Let There Be Carnage,...,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,,Venom: Let There Be Carnage,False,7.2,3708,2021,269.489,269.489
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,None,200000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",https://www.netflix.com/us/title/81161626,512195,tt7991608,en,Red Notice,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Pro and cons.,Red Notice,False,6.8,1616,2021,269.489,269.489
2,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,None,150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.marvel.com/movies/shang-chi-and-th...,566525,tt9376612,en,Shang-Chi and the Legend of the Ten Rings,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,You can't outrun your destiny.,Shang-Chi and the Legend of the Ten Rings,False,7.9,3844,2021,269.489,269.489
3,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,None,9100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,589761,tt10648714,ru,Чернобыль,...,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Chernobyl 1986,False,6.2,214,2021,269.489,269.489
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,None,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",,585245,tt2397461,en,Clifford the Big Red Dog,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Adventure has never been bigger.,Clifford the Big Red Dog,False,7.6,332,2021,269.489,269.489


In [61]:
# Create adjusted column for each feature
features = ["revenue", "budget"]

for feature in features:
    movie_details_df[f'adjusted_{feature}'] = (movie_details_df[f'original_{feature}'] * movie_details_df['cpi_2021']) / movie_details_df['cpi_old']

movie_details_df.head()

,adult,backdrop_path,belongs_to_collection,original_budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,year,cpi_2021,cpi_old,adjusted_revenue,adjusted_budget
0,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"{'id': 558216, 'name': 'Venom Collection', 'po...",110000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.venom.movie,580489,tt7097896,en,Venom: Let There Be Carnage,...,,Venom: Let There Be Carnage,False,7.2,3708,2021,269.489,269.489,454000000.0,110000000.0
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,None,200000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",https://www.netflix.com/us/title/81161626,512195,tt7991608,en,Red Notice,...,Pro and cons.,Red Notice,False,6.8,1616,2021,269.489,269.489,0.0,200000000.0
2,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,None,150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.marvel.com/movies/shang-chi-and-th...,566525,tt9376612,en,Shang-Chi and the Legend of the Ten Rings,...,You can't outrun your destiny.,Shang-Chi and the Legend of the Ten Rings,False,7.9,3844,2021,269.489,269.489,430238384.0,150000000.0
3,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,None,9100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,589761,tt10648714,ru,Чернобыль,...,,Chernobyl 1986,False,6.2,214,2021,269.489,269.489,5370393.0,9100000.0
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,None,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",,585245,tt2397461,en,Clifford the Big Red Dog,...,Adventure has never been bigger.,Clifford the Big Red Dog,False,7.6,332,2021,269.489,269.489,40000000.0,0.0


In [62]:
# Check values
movie_details_df[['original_budget', 'adjusted_budget', 'original_revenue', 'adjusted_revenue', 'year']]

,original_budget,adjusted_budget,original_revenue,adjusted_revenue,year
0,110000000,1.100000e+08,454000000,4.540000e+08,2021
1,200000000,2.000000e+08,0,0.000000e+00,2021
2,150000000,1.500000e+08,430238384,4.302384e+08,2021
3,9100000,9.100000e+06,5370393,5.370393e+06,2021
4,0,0.000000e+00,40000000,4.000000e+07,2021
...,...,...,...,...,...
1965,3000000,2.642049e+07,22182353,1.953562e+08,1963
1966,7000000,3.506362e+07,470653000,2.357543e+09,1975
1967,3180000,4.394744e+07,9644124,1.332813e+08,1946
1968,6000000,5.215916e+07,102272727,8.890766e+08,1964


## Low Budget:

In [63]:
# Create budget bins
bins = [1, 15000000, 50000000, 150000000, 380000000] 
bin_names = ["1 to 15m", "16m to 50m", "51m to 150m", "151m to 380m"]

# Append a budget bin column
movie_details_df["budget_bins"] = pd.cut(movie_details_df["adjusted_budget"], bins, labels=bin_names)
movie_details_df["budget_bins"].value_counts()

51m to 150m     371
16m to 50m      335
151m to 380m    245
1 to 15m        207
Name: budget_bins, dtype: int64

In [64]:
# Low Budget
movie_details_df.loc[movie_details_df["budget_bins"] == "1 to 15m"].head()

,adult,backdrop_path,belongs_to_collection,original_budget,genres,homepage,id,imdb_id,original_language,original_title,...,title,video,vote_average,vote_count,year,cpi_2021,cpi_old,adjusted_revenue,adjusted_budget,budget_bins
3,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,None,9100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,589761,tt10648714,ru,Чернобыль,...,Chernobyl 1986,False,6.2,214,2021,269.489,269.489,5370393.0,9100000.0,1 to 15m
10,False,/4EJSMQOM1bZPHvzqAQe87suBxdf.jpg,None,250000,"[{'id': 10752, 'name': 'War'}, {'id': 36, 'nam...",https://www.netflix.com/title/81450071,885110,tt15847872,en,Amina,...,Amina,False,7.0,34,2021,269.489,269.489,0.0,250000.0,1 to 15m
17,False,/qXctHIfK4LAgHH7qZiJJVCPclaY.jpg,None,45223,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",https://www.disneyplus.com/movies/ciao-alberto...,876716,tt15476620,en,Ciao Alberto,...,Ciao Alberto,False,7.7,222,2021,269.489,269.489,0.0,45223.0,1 to 15m
19,False,/lV3UFPPxDIPelh46G9oySXN9Mcz.jpg,"{'id': 702624, 'name': 'After Collection', 'po...",14000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",,744275,tt13069986,en,After We Fell,...,After We Fell,False,7.2,1054,2021,269.489,269.489,19000000.0,14000000.0,1 to 15m
29,False,/dIibeeq4QMay5bTJ2vjr72IFFRo.jpg,"{'id': 589755, 'name': 'Последний богатырь (Ко...",8400000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",,589754,tt13606158,ru,Последний богатырь: Корень зла,...,The Last Warrior: Root of Evil,False,7.0,66,2021,269.489,269.489,27658331.0,8400000.0,1 to 15m


## Keywords DataFrame:

In [65]:
# Convert keywords to json
json_keywords_string = json.dumps(keywords)
# Convert json to dataframe
movie_keywords_df = pd.read_json(json_keywords_string)

# Export to save
# movie_keywords_df.to_csv("./static/data/movie_keywords.csv", index=False)
movie_keywords_df.head()

,id,keywords
0,580489,"[{'id': 1701, 'name': 'hero'}, {'id': 2095, 'n..."
1,512195,"[{'id': 1812, 'name': 'fbi'}, {'id': 6710, 'na..."
2,566525,"[{'id': 779, 'name': 'martial arts'}, {'id': 9..."
3,589761,"[{'id': 210326, 'name': 'chernobyl'}]"
4,585245,"[{'id': 818, 'name': 'based on novel or book'}..."


## Credits DataFrame:

In [66]:
# Convert credits to json
json_credits_string = json.dumps(credits)
# Convert json to dataframe
movie_credits_df = pd.read_json(json_credits_string)

# Export to save
# movie_credits_df.to_csv("./static/data/movie_credits.csv", index=False)
movie_credits_df.head()

,id,cast,crew
0,580489,"[{'adult': False, 'gender': 2, 'id': 2524, 'kn...","[{'adult': False, 'gender': 2, 'id': 149, 'kno..."
1,512195,"[{'adult': False, 'gender': 2, 'id': 18918, 'k...","[{'adult': False, 'gender': 2, 'id': 9543, 'kn..."
2,566525,"[{'adult': False, 'gender': 2, 'id': 1489211, ...","[{'adult': False, 'gender': 1, 'id': 7232, 'kn..."
3,589761,"[{'adult': False, 'gender': 2, 'id': 562730, '...","[{'adult': False, 'gender': 1, 'id': 30247, 'k..."
4,585245,"[{'adult': False, 'gender': 1, 'id': 1696017, ...","[{'adult': False, 'gender': 1, 'id': 8220, 'kn..."


## Certifications DataFrame:

In [67]:
# Convert certifications to json
json_certifications_string = json.dumps(certifications)
# Convert json to dataframe
movie_certifications_df = pd.read_json(json_certifications_string)

# Export to save
# movie_certifications_df.to_csv("./static/data/movie_certifications.csv", index=False)
movie_certifications_df.head()

,id,results
0,580489,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
1,512195,"[{'iso_3166_1': 'IT', 'release_dates': [{'cert..."
2,566525,"[{'iso_3166_1': 'TR', 'release_dates': [{'cert..."
3,589761,"[{'iso_3166_1': 'LT', 'release_dates': [{'cert..."
4,585245,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."


In [68]:
# Extract certification information and append to new column
movie_certifications_df['certification'] = movie_certifications_df['results'].apply(extract_certification)
movie_certifications_df = movie_certifications_df[['id', 'certification']]

In [69]:
movie_certifications_df.head()

,id,certification
0,580489,PG-13
1,512195,PG-13
2,566525,PG-13
3,589761,
4,585245,PG


In [70]:
# Convert ids to ints (to merge correctly)
movie_keywords_df['id'] = movie_keywords_df['id'].astype('int')
movie_details_df['id'] = movie_details_df['id'].astype('int')
movie_credits_df['id'] = movie_credits_df['id'].astype('int')
movie_certifications_df['id'] = movie_certifications_df['id'].astype('int')

In [71]:
# Merge keywords with details
movie_df = movie_details_df.merge(movie_keywords_df, on='id')
movie_df = movie_df.merge(movie_credits_df, on='id')
movie_df = movie_df.merge(movie_certifications_df, on='id')

# Export to save
# movie_df.to_csv("./static/data/movies_merged.csv", index=False)

In [72]:
movie_df.head(20)

,adult,backdrop_path,belongs_to_collection,original_budget,genres,homepage,id,imdb_id,original_language,original_title,...,year,cpi_2021,cpi_old,adjusted_revenue,adjusted_budget,budget_bins,keywords,cast,crew,certification
0,False,/70nxSw3mFBsGmtkvcs91PbjerwD.jpg,"{'id': 558216, 'name': 'Venom Collection', 'po...",110000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.venom.movie,580489,tt7097896,en,Venom: Let There Be Carnage,...,2021,269.489,269.489,454000000.0,110000000.0,51m to 150m,"[{'id': 1701, 'name': 'hero'}, {'id': 2095, 'n...","[{'adult': False, 'gender': 2, 'id': 2524, 'kn...","[{'adult': False, 'gender': 2, 'id': 149, 'kno...",PG-13
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,None,200000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",https://www.netflix.com/us/title/81161626,512195,tt7991608,en,Red Notice,...,2021,269.489,269.489,0.0,200000000.0,151m to 380m,"[{'id': 1812, 'name': 'fbi'}, {'id': 6710, 'na...","[{'adult': False, 'gender': 2, 'id': 18918, 'k...","[{'adult': False, 'gender': 2, 'id': 9543, 'kn...",PG-13
2,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,None,150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.marvel.com/movies/shang-chi-and-th...,566525,tt9376612,en,Shang-Chi and the Legend of the Ten Rings,...,2021,269.489,269.489,430238384.0,150000000.0,51m to 150m,"[{'id': 779, 'name': 'martial arts'}, {'id': 9...","[{'adult': False, 'gender': 2, 'id': 1489211, ...","[{'adult': False, 'gender': 1, 'id': 7232, 'kn...",PG-13
3,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,None,9100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,589761,tt10648714,ru,Чернобыль,...,2021,269.489,269.489,5370393.0,9100000.0,1 to 15m,"[{'id': 210326, 'name': 'chernobyl'}]","[{'adult': False, 'gender': 2, 'id': 562730, '...","[{'adult': False, 'gender': 1, 'id': 30247, 'k...",
4,False,/zBkHCpLmHjW2uVURs5uZkaVmgKR.jpg,None,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",,585245,tt2397461,en,Clifford the Big Red Dog,...,2021,269.489,269.489,40000000.0,0.0,NaN,"[{'id': 818, 'name': 'based on novel or book'}...","[{'adult': False, 'gender': 1, 'id': 1696017, ...","[{'adult': False, 'gender': 1, 'id': 8220, 'kn...",PG
5,False,/g2djzUqA6mFplzC03gDk0WSyg99.jpg,None,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",https://movies.disney.com/encanto,568124,tt2953050,en,Encanto,...,2021,269.489,269.489,69000000.0,0.0,NaN,[],"[{'adult': False, 'gender': 1, 'id': 968367, '...","[{'adult': False, 'gender': 0, 'id': 8159, 'kn...",
6,False,/vIPIyTJqcgOKgKcExCvTDpLpTYW.jpg,"{'id': 531241, 'name': 'Spider-Man (Avengers) ...",0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.spidermannowayhome.movie,634649,tt10872600,en,Spider-Man: No Way Home,...,2021,269.489,269.489,0.0,0.0,NaN,"[{'id': 1701, 'name': 'hero'}, {'id': 5451, 'n...","[{'adult': False, 'gender': 2, 'id': 1136406, ...","[{'adult': False, 'gender': 1, 'id': 2519, 'kn...",PG-13
7,False,/uWGPC7j70LE64nbetxQGSSYJO53.jpg,None,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 10752,...",,762433,tt13432484,en,Zeros and Ones,...,2021,269.489,269.489,0.0,0.0,NaN,"[{'id': 588, 'name': 'rome, italy'}, {'id': 94...","[{'adult': False, 'gender': 2, 'id': 569, 'kno...","[{'adult': False, 'gender': 1, 'id': 21342, 'k...",R
8,False,/r2GAjd4rNOHJh6i6Y0FntmYuPQW.jpg,"{'id': 645, 'name': 'James Bond Collection', '...",242000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.007.com/no-time-to-die/,370172,tt2382320,en,No Time to Die,...,2021,269.489,269.489,758000000.0,242000000.0,151m to 380m,"[{'id': 470, 'name': 'spy'}, {'id': 156095, 'n...","[{'adult': False, 'gender': 2, 'id': 8784, 'kn...","[{'adult': False, 'gender': 2, 'id': 947, 'kno...",PG-13
9,False,/oE6bhqqVFyIECtBzqIuvh6JdaB5.jpg,None,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://tv.apple.com/movie/umc.cmc.47dkj9f2ho3...,522402,tt3420504,en,Finch,...,2021,269.489,269.489,0.0,0.0,NaN,"[{'id': 14544, 'name': 'robot'}, {'id': 189092...","[{'adult': 

## Call Functions to Extract Details:

In [73]:
# Director
movie_df['director'] = movie_df['crew'].apply(get_director)

# Director Gender
movie_df['director_gender'] = movie_df['crew'].apply(get_director_gender)

# Producers
movie_df['producers'] = movie_df['crew'].apply(get_producers)

# Screenplay writers
movie_df['writers'] = movie_df['crew'].apply(get_writers)

# Cast
movie_df['cast'] = movie_df['cast'].apply(get_cast_list)

# Production Company Country of Origin
movie_df['production_company_origin'] = movie_df['production_companies'].apply(get_production_company_country)

# Spoken Languages in Movie
movie_df['spoken_languages']  = movie_df['spoken_languages'].apply(get_languages)

# Create foreign language column for if the original language was not English
movie_df['foreign_language'] = movie_df['original_language'].apply(original_language_binary)

# Get List: Keywords, genres, & production countries
features = ['keywords', 'genres', 'production_companies']

for feature in features:
    movie_df[feature] = movie_df[feature].apply(get_list)

In [74]:
# Show the results
movie_df[['title', 'cast', 'director', 'director_gender', 'producers', 'writers', 'keywords', 'genres', 'production_companies', 'production_company_origin', 'spoken_languages', 'original_language', 'foreign_language']].head()

,title,cast,director,director_gender,producers,writers,keywords,genres,production_companies,production_company_origin,spoken_languages,original_language,foreign_language
0,Venom: Let There Be Carnage,"[Tom Hardy, Woody Harrelson, Michelle Williams...",[Andy Serkis],[2],"[Tom Hardy, K.C. Hodenfield, Avi Arad, Jonatha...","[Tom Hardy, Todd McFarlane, Kelly Marcel, Kell...","[hero, anti hero, villain, sequel, superhero, ...","[Science Fiction, Action, Adventure]","[Marvel Entertainment, Pascal Pictures, Columb...","[US, US, US, US, CN]","[Spanish, English]",en,0
1,Red Notice,"[Dwayne Johnson, Ryan Reynolds, Gal Gadot, Rit...",[Rawson Marshall Thurber],[2],"[Beau Flynn, Dwayne Johnson, Rawson Marshall T...",[Rawson Marshall Thurber],"[fbi, interpol, heist, art thief]","[Action, Comedy, Crime, Thriller]","[Flynn Picture Company, Seven Bucks Production...","[US, US, US, ]","[English, Italian, Russian]",en,0
2,Shang-Chi and the Legend of the Ten Rings,"[Simu Liu, Tony Leung Chiu-wai, Awkwafina, Men...",[Destin Daniel Cretton],[2],"[Kevin Feige, Charles Newirth, Louis D'Esposit...","[Dave Callaham, Dave Callaham, Destin Daniel C...","[martial arts, superhero, based on comic, mixe...","[Action, Adventure, Fantasy]",[Marvel Studios],[US],"[English, Mandarin]",en,0
3,Chernobyl 1986,"[Danila Kozlovsky, Oksana Akinshina, Philip Av...",[Danila Kozlovsky],[2],"[Danila Kozlovsky, Danila Kozlovsky, Alexandr ...","[Alexey Kazakov, Elena Ivanova]",[chernobyl],"[Drama, History, Adventure]","[Non-Stop Productions, DK Entertainment, Centr...","[RU, RU, RU, RU]",[Russian],ru,1
4,Clifford the Big Red Dog,"[Darby Camp, Jack Whitehall, Izaac Wang, John ...",[Walt Becker],[2],"[Brad Fischer, Deborah Forte, Deborah Forte, B...","[David Ronn, Jay Scherick, Norman Bridwell, An...","[based on novel or book, giant dog]","[Animation, Comedy, Family]","[Paramount, Entertainment One, Kerner Entertai...","[US, CA, , US, US, US]",[English],en,0


In [75]:
# Drop NA from selected columns
movie_df.dropna(inplace=True, how="any", subset=['genres', 'production_companies', 'keywords', 'cast'])
movie_df.dropna(inplace=True, how="any", subset=['director', 'producers', 'writers'])

In [76]:
len(movie_df)

1683

In [77]:
# Clean Data: Convert all features to lowercase and remove spaces
features = ['cast', 'director', 'producers', 'writers', 'keywords', 'genres', 'production_companies']

for feature in features:
    movie_df[f"{feature}_cleaned"] = movie_df[feature].apply(clean_data)

In [78]:
# Call the clean overview function
movie_df['overview_cleaned'] = movie_df['overview'].apply(clean_overview)

In [79]:
# Create soup columns
movie_df['soup'] = movie_df.apply(create_soup, axis = 1)
movie_df['soup_overview'] = movie_df.apply(create_soup_overview, axis = 1)

# Export to save
# movie_df.to_csv("./static/data/movies_cleaned_soup.csv", index=False)

In [80]:
movie_df[['soup']].head(5)

,soup
0,hero antihero villain sequel superhero basedon...
1,fbi interpol heist artthief dwaynejohnson ryan...
2,martialarts superhero basedoncomic mixedmartia...
3,chernobyl danilakozlovsky oksanaakinshina phil...
4,basedonnovelorbook giantdog darbycamp jackwhit...


## Configuration for Posters:

In [81]:
configuration_url = f"https://api.themoviedb.org/3/configuration?api_key={api_key}"
config_response = requests.get(configuration_url).json()
# config_response

In [82]:
# Get images structure
images_url = config_response['images']
# Get Base URL
secure_base_url = images_url['secure_base_url']
# secure_base_url

In [83]:
# Get the size of poster: 2: w185px, 4: w500px, 5: 780px
# images_url['poster_sizes']
poster_size = images_url['poster_sizes'][5]
# poster_size

In [84]:
# Copy poster paths to new df
poster_df = movie_df[['poster_path']].copy()

In [85]:
# Create column with full image path for posters
poster_df['poster_url'] = secure_base_url + poster_size + poster_df['poster_path']

In [86]:
# Export to separate csv
# poster_df.to_csv("./static/data/poster_path.csv", index=False)
poster_df.head()

,poster_path,poster_url
0,/rjkmN1dniUHVYAtwuV3Tji7FsDO.jpg,https://image.tmdb.org/t/p/w780/rjkmN1dniUHVYA...
1,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,https://image.tmdb.org/t/p/w780/wdE6ewaKZHr62b...
2,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,https://image.tmdb.org/t/p/w780/1BIoJGKbXjdFDA...
3,/kfQJQWFEoWRVBH8FUKnT0HX1yRS.jpg,https://image.tmdb.org/t/p/w780/kfQJQWFEoWRVBH...
4,/ygPTrycbMSFDc5zUpy4K5ZZtQSC.jpg,https://image.tmdb.org/t/p/w780/ygPTrycbMSFDc5...


In [87]:
# Create column with full image path for posters (in real df this time)
movie_df['poster_url'] = secure_base_url + poster_size + movie_df['poster_path']

# TESTING

## Export CSV:

In [88]:
# Create a lowercase column for easier search
movie_df["lowercase_title"] = movie_df['title'].apply(lambda x: x.lower())

In [89]:
# Save file - used for calling for information
movie_df.to_csv("../static/data/movie_db.csv", index=False)

In [ ]:
movie_df.head()

# Machine Learning Recommender:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Use Count Vectorizer to create counts for each word
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movie_df['soup_overview'])

In [ ]:
count_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset the index
# movie_df = movie_df.reset_index()
# Create series with index & titles of movies
indices = pd.Series(movie_df.index, index=movie_df['title']).drop_duplicates()

In [ ]:
movie_df.head()

## Function to Recommend Titles:

In [ ]:
def get_similarity_scores(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the most similar movies
    sim_scores = sim_scores[1:11]
#     print(sim_scores)
    
    # Convert list to DataFrame
    sim_scores_df = pd.DataFrame(sim_scores, columns = ["index", "similarity_score"])
    
    # Return top 10 most similar scores
    return sim_scores_df

In [ ]:
def get_recommendations(original_df, score_df):
    original_df = original_df.merge(score_df, on="index")
    original_df = original_df.sort_values("similarity_score", ascending=False)
    
    return original_df

In [ ]:
# Test out function
movie_title = "Get Out"
similarity_scores_df = get_similarity_scores(movie_title, cosine_sim)

In [ ]:
# Call Get Recommendations Function
recommendations = get_recommendations(movie_df, similarity_scores_df)

In [ ]:
# Print out dataframe
# recommendations[['title', 'similarity_score']].to_csv("./soup_test_3.csv")
recommendations[['title', 'similarity_score', 'id']].head(10)

## Database:

In [ ]:
from sqlalchemy import create_engine, inspect

In [ ]:
# configure the connection string
rds_connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
        
# connect to the database
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [ ]:
engine.execute("DROP TABLE movies")

In [ ]:
# Append data to table
movie_df.to_sql(name='movies', con=conn, if_exists='append', index=False)

In [ ]:
# Use inspector to find table names
Inspector = inspect(engine)
Inspector.get_table_names()

In [ ]:
# Check movies table
pd.read_sql_query('select * from movies', con=conn).head()